# Applying Indicators to Price Data
Takes the output from `iq_feed_cleaning.ipynb` and for each row, adds indicators. \
This is preferred over calculating the indicators for each timestamp/date, as it saves 
a lot of computations. \
It does require us to do a look-up, but so does the alternative.

In [1]:
import pandas as pd
import pytz
eastern = pytz.timezone('US/Eastern')
from src.config import config
from concurrent.futures import ThreadPoolExecutor
import os
from src.utils.tickers import get_tickers

g:\Meine Ablage\NewsTrading\trading_bot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tickers = get_tickers(config.data.iqfeed.daily.cleaned)

In [5]:
# Beta calculatiopn (use intraday?)
spy = pd.read_parquet(path=f"{config.data.iqfeed.daily.cleaned}/SPY_daily.parquet")
prices = pd.read_parquet(path=f"{config.data.iqfeed.daily.cleaned}/AAPL_daily.parquet")
X = pd.merge(prices, spy, left_index=True, right_index=True, suffixes=("_stock", "_SPY"))

In [24]:
X["r_stock"] = X["adj_close_stock"]/X["adj_close_stock"].shift() - 1 
X["r_SPY"] = X["adj_close_SPY"]/X["adj_close_SPY"].shift() - 1 
returns = X[["r_stock", "r_SPY"]].dropna()

In [25]:
from arch import arch_model
# define lists for storing objects
coeffs = []
cond_vol = []
std_resids = []
models = []

In [26]:
for asset in returns.columns:
    model = arch_model(returns[asset], mean = 'Constant', vol = 'GARCH', p = 1, o = 0, q = 1).fit(update_freq = 0,
                                                                                                    disp = 'off')
    coeffs.append(model.params)
    cond_vol.append(model.conditional_volatility)
    std_resids.append(model.resid / model.conditional_volatility)
    models.append(model)

g:\Meine Ablage\NewsTrading\trading_bot\.venv\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0003107. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
g:\Meine Ablage\NewsTrading\trading_bot\.venv\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001177. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
g:\Meine Ablage\NewsTrading\tradin

In [35]:
# store the results in df
coeffs_df = pd.DataFrame(coeffs, index=returns.columns)
cond_vol_df = pd.DataFrame(cond_vol).transpose().set_axis(returns.columns, 
                                                          axis = 'columns')
std_resids_df = pd.DataFrame(std_resids).transpose().set_axis(returns.columns,
                                                             axis = 'columns')

In [36]:
# calculate the constant conditional correlation matrix (CCC) R:

R = std_resids_df.transpose().dot(std_resids_df).div(len(std_resids_df))

In [37]:
# calculate one step ahead forecastof the conditional covariance matrix
import numpy as np
diag = []
D = np.zeros((2, 2))

for model in models:
    diag.append(model.forecast(horizon = 1).variance.values[-1][0])
    
diag = np.sqrt(np.array(diag))
np.fill_diagonal(D, diag)

H = np.matmul(np.matmul(D, R.values), D)

In [38]:
H

array([[1.87133148e-04, 6.89123094e-04],
       [6.89123094e-04, 5.86605807e+00]])

------------

In [24]:
def add_indicators(ticker):
    prices = pd.read_parquet(path=f"{config.data.iqfeed.daily.cleaned}/{ticker}_daily.parquet")
    prices["std_252"] = prices["adj_close"].pct_change().rolling(252, min_periods=252).std()*(252**0.5)
    prices["dollar_volume"] = prices["adj_volume"] * (prices["adj_close"] + prices["adj_open"])/2
    prices["r_intra_(t-1)"] = (prices["adj_close"] / prices["adj_open"] - 1).shift(periods=1)
    prices["unadj_open"] = prices["adj_open"] / prices["cum_split_ratio"]
    prices.to_parquet(path=f"{config.data.iqfeed.daily.cleaned}/{ticker}_daily.parquet")

In [25]:
pool_obj = ThreadPoolExecutor(max_workers=os.cpu_count()-1)
ans = pool_obj.map(add_indicators, tickers)
result = list(ans)